## Parte 1

hay rocas redondas O, rocas cuadradas # y espacios vacíos ., al inclinar la plataforma las rocas redondas girarán en el sentido indicado. Hay que calcular la carga total en el lado norte. Para ello se suman las rocas redondas y cada roca genera una carga igual a la distancia de la roca hasta el punto sur de la plataforma.

In [1]:
import numpy as np

In [3]:
def get_input(filename):
    rock_map = []
    with open(filename) as file:
        for line in file:
            symbols = [s for s in line.strip()]
            rock_map.append(symbols)
    return np.array(rock_map)

In [4]:
def tilt_column(column):
    movements = True
    while movements:
        movements = False
        for i in range(1, len(column)):
            if column[i - 1] == '.' and column[i] == 'O':
                column[i - 1] = 'O'
                column[i] = '.'
                movements = True
    return column

In [5]:
def tilt_map(rock_map):
    w = rock_map.shape[1]
    for i in range(w):
        tilt_column(rock_map[:, i])
    return rock_map

In [6]:
def weigh_rocks(tilted_map):
    h = tilted_map.shape[0]
    w = tilted_map.shape[1]
    
    weight = 0
    for i in range(h):
        rocks = np.count_nonzero(tilted_map[i, :] == 'O')
        weight += rocks * (h - i)
    return weight

In [7]:
rock_array = get_input('test.txt')
tilted_map = tilt_map(rock_array)
weight = weigh_rocks(tilted_map)
weight

136

In [8]:
rock_array = get_input('input.txt')
tilted_map = tilt_map(rock_array)
weight = weigh_rocks(tilted_map)
weight

105003

## Parte 2

un ciclo consiste en girar norte, oeste, sur y este, después de cada ciclo las rocas redondeadas se mueven antes de que la plataforma gire en la siguiente dirección. El ciclo se repite 1_000_000_000 veces. Cual es la carga total en el norte al finalizar?

In [9]:
def tilt_chunk(chunk, direction):
    movements = True
    while movements:
        movements = False
        if direction == -1:
            for i in range(1, len(chunk)):
                if chunk[i - 1] == '.' and chunk[i] == 'O':
                    chunk[i - 1] = 'O'
                    chunk[i] = '.'
                    movements = True
        elif direction == 1:
            for i in range(len(chunk) - 2, -1, -1):
                if chunk[i + 1] == '.' and chunk[i] == 'O':
                    chunk[i + 1] = 'O'
                    chunk[i] = '.'
                    movements = True
    return chunk

In [15]:
def tilt_mapd(rock_map, direction):
    h = rock_map.shape[0]
    w = rock_map.shape[1]
    
    if direction == 'n':
        for i in range(w):
            tilt_chunk(rock_map[:, i], -1)
    elif direction == 's':
        for i in range(w):
            tilt_chunk(rock_map[:, i], 1)
    elif direction == 'w':
        for i in range(h):
            tilt_chunk(rock_map[i, :], -1)
    elif direction == 'e':
        for i in range(h):
            tilt_chunk(rock_map[i, :], 1)
    return rock_map

In [16]:
def roll(rock_map):
    directions = ['n', 'w', 's', 'e']
    
    for d in directions:
        tilt_mapd(rock_map, d)
    return rock_map

In [19]:
def tilt_ntimes(rock_array, ntimes):
    cache = {}
    found = False
    first = True
    count = 0
    
    while count < ntimes:
        roll(rock_array)
        count += 1
        
        map_bytes = rock_array.tobytes()
        found = map_bytes in cache
        
        if found and first:
            size = count - cache[map_bytes]
            print(f'longitud del bucle: {size}')
            print(f'antes de que empiece el bucle: {count}')
            
            count += size * ((ntimes - count) // size)
            first = False
        elif not found:
            cache[map_bytes] = count
    return rock_array

In [20]:
rock_array = get_input('test.txt')
rock_array = tilt_ntimes(rock_array, 1_000_000_000)
weigh_rocks(rock_array)

longitud del bucle: 7
antes de que empiece el bucle: 10


64

In [21]:
rock_array = get_input('input.txt')
rock_array = tilt_ntimes(rock_array, 1_000_000_000)
weigh_rocks(rock_array)

longitud del bucle: 51
antes de que empiece el bucle: 132


93742